In [114]:
import random
import numpy as np
import math

# Task 1

In [115]:
class Individual:
    def __init__(self, n, bit_values):
        self.n = n
        self.bit_values = bit_values
    
    def mutate(self):
        mutated_bit_values = self.bit_values.copy()
        k = np.random.binomial(self.n, 1/self.n)
        indices = random.sample(range(self.n), k)
        for i in indices:
            mutated_bit_values[i] = 1 - mutated_bit_values[i]
        return Individual(self.n, mutated_bit_values)
    
    
# def flip_bits_randomly(bits, p):   (geometric distribution version)
#     n = len(bits)
#     i = 0
#     while i < n:
#         # geometric distribution: number of trials until success
#         # geometric(p) can be sampled via inverse CDF
#         skip = int(math.log(1 - random.random()) / math.log(1 - p))

#         i += skip
#         if i >= n:
#             break

#         bits[i] ^= 1   # flip the bit
#         i += 1

In [116]:
class Frequency_Vector :
    def __init__(self, n, values=None): 
        self.n = n
        if values is None:
            self.values = [1/n]*n
        else :
            self.values = []
            for v in values:
                v = max(min(v,1-1/n),1/n)
                self.values.append(v)

    
    def update(self, i, p_i):
        self.values[i] = max(min(p_i,1-1/self.n),1/self.n)
        
    # def get(self, i):
    #     return self.values[i]
    
    def generate_individual(self):
        bit_values = []
        for i in range(self.n):
            r = random.random()
            if r < self.values[i]:
                bit_values.append(1)
            else:
                bit_values.append(0)
        return Individual(self.n, bit_values)
    
    

# Task 2

In [117]:
def sig(p, H, eps, n):
    m = H[0]
    limit = m*p+eps*max(np.sqrt(m*p*np.log(n)), np.log(n))
    if limit <= H[1] and p in [1/n,1/2]:
        return 1 # UP
    elif limit <= H[0] - H[1] and p in [1-1/n,1/2]:
        return -1 # DOWN
    else:
        return 0 # STAY
    

In [118]:
class Simplified_History :
    def __init__(self, size=0, zeros=0, ones=0):
        self.size = size
        self.zeros = zeros
        self.ones = ones
    
    def add(self, ind, n):
        self.size+=1
        if ind == 1:
            self.ones+=1
        else:
            self.zeros+=1    
    
    def count_ones(self):
        return self.ones
    
    def count_zeros(self):
        return self.zeros
    
    def get_subsequences(self):
        return [[self.size, self.ones]]
    
    def reset(self):
        self.size = 0
        self.zeros = 0
        self.ones = 0

In [119]:
class Original_History :
    def __init__(self, size=0, ones=0, next=None):
        self.size = size
        self.ones = ones        
        self.next = next        
    
    def count_ones(self):
        return self.ones
    
    def count_zeros(self):
        return self.size - self.ones
    
    def add(self, ind, n):
        if self.size < math.log(n):
            self.size +=1
            if ind == 1:
                self.ones +=1
        else:
            self = Original_History(1, ind, self)
            consolidate(self)
            
    def get_subsequences(self):
        subsequences = []
        curr = self
        while curr is not None :
            subsequences.append([curr.size + subsequences[-1][0] if subsequences else curr.size, curr.ones + subsequences[-1][1] if subsequences else curr.ones])
            curr = curr.next
        return subsequences
    
    def reset(self):
        self.size = 0
        self.ones = 0
        self.next = None
    
def consolidate(L):
    curr = L
    if curr is None:
        next = curr.next
        alreadySeenDouble = False
        while next is not None:
            if curr.size == next.size:
                if alreadySeenDouble:
                    m = Original_History(curr.size + next.size, curr.ones + next.ones, curr.next)
                    curr = m
                    curr.next = next.next
                    next = curr.next
                    alreadySeenDouble = False
                else:
                    alreadySeenDouble = True
                    curr = next
            else:
                curr = next     

The maximum number of merges is $O(log(m))$ where $m$ is the number of bits added so far, this happens when each block size doubles, for example: 

# Task 3

In [120]:
def sig_cga(f, termination_condition, n, history_type, eps):
    t = 0
    x_1 = None # have to initialize x_1 for the termination condition
    p = Frequency_Vector(n, [0.5]*n)
    if history_type == "simplified":
        H = [Simplified_History() for _ in range(n+1)]
    else:
        H = [Original_History() for _ in range(n+1)]
    while not termination_condition(t, x_1):  # have to depend on an individual ?
        x_1 = p.generate_individual()
        x_2 = p.generate_individual()
        if f(x_1) < f(x_2):
            x_1, x_2 = x_2, x_1
        for i in range(n):
            H[i].add(x_1.bit_values[i], n)
            for h in H[i].get_subsequences():
                s = sig(p.values[i], h, eps, n)
                if s == 1:
                    p.update(i, 1-1/n)
                elif s == -1:
                    p.update(i, 1/n)
                if s != 0:
                    H[i].reset()
                    break
        t+=1
    return x_1,t

# Task 4

In [121]:
def cga(K, n, f, termination_criterion):
    t = 0
    x_1 = None
    p = Frequency_Vector(n, [0.5]*n)
    while not termination_criterion(t, x_1):
        x_1 = p.generate_individual()
        x_2 = p.generate_individual()
        if f(x_1) < f(x_2):
            x_1, x_2 = x_2, x_1
        for i in range(n):
            p.update(i, p.values[i] + 1/K * (x_1.bit_values[i] - x_2.bit_values[i]))
        t+=1
    return x_1,t

def one_plus_one_ea(f, termination_criterion, n):
    t = 0
    x = Individual(n, [random.randint(0, 1) for _ in range(n)])
    while not termination_criterion(t, x):
        y = x.mutate()
        if f(y) > f(x):
            x = y
        t+=1
    return x,t

# Task 5

In [122]:
def OneMax(ind):
    return sum(ind.bit_values)

def LeadingOnes(ind):
    count = 0
    for bit in ind.bit_values:
        if bit == 1:
            count += 1
        else:
            break
    return count

def Jump(ind, k):
    n = ind.n
    ones = sum(ind.bit_values)
    if ones == n or ones <= n-k:
        return k + ones
    else:
        return n - ones

eval = 1000

def termination_criterion(t, x):
    if x is None:
        return False
    return t >= eval or x.bit_values == [1]*x.n

# Task 6

In [123]:
n_tries = 10

## One Max

In [124]:
np.random.seed(42)

# fixing parameters for experiments
n_s = [100, 300]
eps_s = [12, 9]   # for sig_cga
def K_s(n):
    K_0 = np.sqrt(n * np.log(n))
    return [0.25*K_0, K_0, 2*K_0]
T = 3000

# sig_cga for simplified history results
results_sig_cga_simplified_One_Max = {}
for n in n_s:
    for eps in eps_s:
        f_values = []
        success_count = 0
        t_s = []
        eval = T//2
        for _ in range(n_tries):
            x, t = sig_cga(OneMax, termination_criterion, n, "simplified", eps)
            f_values.append(OneMax(x))
            if x.bit_values == [1]*n:
                success_count += 1
                t_s.append(t)
        results_sig_cga_simplified_One_Max[(n, eps)] = (success_count/10, np.mean(t_s) if success_count > 0 else None,
                                                np.mean(f_values), np.std(f_values))
        
# sig_cga for original history results
results_sig_cga_original_One_Max = {}
for n in n_s:
    for eps in eps_s:
        f_values = []
        success_count = 0
        t_s = []
        eval = T//2
        for _ in range(n_tries):
            x, t = sig_cga(OneMax, termination_criterion, n, "original", eps)
            f_values.append(OneMax(x))
            if x.bit_values == [1]*n:
                success_count += 1
                t_s.append(t)
        results_sig_cga_original_One_Max[(n, eps)] = (success_count/10, np.mean(t_s) if success_count > 0 else None,
                                                np.mean(f_values), np.std(f_values))
        
# cga results
results_cga_One_Max = {}
for n in n_s:
    for K in K_s(n):
        f_values = []
        success_count = 0
        t_s = []
        eval = T//2
        for _ in range(n_tries):
            x, t = cga(K, n, OneMax, termination_criterion)
            f_values.append(OneMax(x))
            if x.bit_values == [1]*n:
                success_count += 1
                t_s.append(t)
        results_cga_One_Max[(n, K)] = (success_count/10, np.mean(t_s) if success_count > 0 else None,
                                np.mean(f_values), np.std(f_values))
        
# one_plus_one_ea results
results_one_plus_one_ea_One_Max = {}
for n in n_s:
    f_values = []
    success_count = 0
    t_s = []
    eval = T
    for _ in range(n_tries):
        x, t = one_plus_one_ea(OneMax, termination_criterion, n)
        f_values.append(OneMax(x))
        if x.bit_values == [1]*n:
            success_count += 1
            t_s.append(t)
    results_one_plus_one_ea_One_Max[n] = (success_count/10, np.mean(t_s) if success_count > 0 else None,
                                  np.mean(f_values), np.std(f_values))
    


        

## Leading Ones

In [125]:
np.random.seed(42)

n_s = [100, 200]
eps_s = [12, 9]
def K_s(n):
    K_0 = n * np.log(n)**2
    return [0.25*K_0, K_0, 2*K_0]
T = 3000

# sig_cga for simplified history results
results_sig_cga_simplified_LeadingOnes = {}
for n in n_s:
    for eps in eps_s:
        f_values = []
        success_count = 0
        t_s = []
        evals = T//2
        for _ in range(n_tries):
            x, t = sig_cga(LeadingOnes, termination_criterion, n, "simplified", eps)
            f_values.append(LeadingOnes(x))
            if x.bit_values == [1]*n:
                success_count += 1
                t_s.append(t)
        results_sig_cga_simplified_LeadingOnes[(n, eps)] = (success_count/10, np.mean(t_s) if success_count > 0 else None,
                                                np.mean(f_values), np.std(f_values))
        
# sig_cga for original history results
results_sig_cga_original_LeadingOnes = {}
for n in n_s:
    for eps in eps_s:
        f_values = []
        success_count = 0
        t_s = []
        evals = T//2
        for _ in range(n_tries):
            x, t = sig_cga(LeadingOnes, termination_criterion, n, "original", eps)
            f_values.append(LeadingOnes(x))
            if x.bit_values == [1]*n:
                success_count += 1
                t_s.append(t)
        results_sig_cga_original_LeadingOnes[(n, eps)] = (success_count/10, np.mean(t_s) if success_count > 0 else None,
                                                np.mean(f_values), np.std(f_values))
        
# cga results
results_cga_LeadingOnes = {}
for n in n_s:
    for K in K_s(n):
        f_values = []
        success_count = 0
        t_s = []
        evals = T//2
        for _ in range(n_tries):
            x, t = cga(K, n, LeadingOnes, termination_criterion)
            f_values.append(LeadingOnes(x))
            if x.bit_values == [1]*n:
                success_count += 1
                t_s.append(t)
        results_cga_LeadingOnes[(n, K)] = (success_count/10, np.mean(t_s) if success_count > 0 else None,
                                np.mean(f_values), np.std(f_values))
        
# one_plus_one_ea results
results_one_plus_one_ea_LeadingOnes = {}
for n in n_s:
    f_values = []
    success_count = 0
    t_s = []
    evals = T
    for _ in range(10):
        x, t = one_plus_one_ea(LeadingOnes, termination_criterion, n)
        f_values.append(LeadingOnes(x))
        if x.bit_values == [1]*n:
            success_count += 1
            t_s.append(t)
    results_one_plus_one_ea_LeadingOnes[n] = (success_count/10, np.mean(t_s) if success_count > 0 else None,
                                  np.mean(f_values), np.std(f_values))

## Jumping Ones

In [126]:
np.random.seed(42)

k_s = [2, 3]
n_s = [100, 300]
eps_s = [12, 9]
def K_s(n):
    K_0 = np.sqrt(n * np.log(n))
    return [0.25*K_0, K_0, 2*K_0]
T = 3000

# sig_cga for simplified history results
results_sig_cga_simplified_Jump = {}
for n in n_s:
    for eps in eps_s:
        for k in k_s:
            f_values = []
            success_count = 0
            t_s = []
            eval = T//2
            for _ in range(n_tries):
                x, t = sig_cga(lambda x : Jump(x, k), termination_criterion, n, "simplified", eps)
                f_values.append(Jump(x, k))
                if x.bit_values == [1]*n:
                    success_count += 1
                    t_s.append(t)
            results_sig_cga_simplified_Jump[(n, eps, k)] = (success_count/10, np.mean(t_s) if success_count > 0 else None,
                                                    np.mean(f_values), np.std(f_values))
        
# sig_cga for original history results
results_sig_cga_original_Jump = {}
for n in n_s:
    for eps in eps_s:
        for k in k_s:
            f_values = []
            success_count = 0
            t_s = []
            eval = T//2
            for _ in range(n_tries):
                x, t = sig_cga(lambda x : Jump(x, k), termination_criterion, n, "original", eps)
                f_values.append(Jump(x, k))
                if x.bit_values == [1]*n:
                    success_count += 1
                    t_s.append(t)
            results_sig_cga_original_Jump[(n, eps, k)] = (success_count/10, np.mean(t_s) if success_count > 0 else None,
                                                    np.mean(f_values), np.std(f_values))

# cga results
results_cga_Jump = {}
for n in n_s:
    for K in K_s(n):
        for k in k_s:
            f_values = []
            success_count = 0
            t_s = []
            eval = T//2
            for _ in range(n_tries):
                x, t = cga(K, n, lambda x : Jump(x, k), termination_criterion)
                f_values.append(Jump(x, k))
                if x.bit_values == [1]*n:
                    success_count += 1
                    t_s.append(t)
            results_cga_Jump[(n, K, k)] = (success_count/10, np.mean(t_s) if success_count > 0 else None,
                                        np.mean(f_values), np.std(f_values))
            
# one_plus_one_ea results
results_one_plus_one_ea_Jump = {}
for n in n_s:
    for k in k_s:
        f_values = []
        success_count = 0
        t_s = []
        eval = T
        for _ in range(n_tries):
            x, t = one_plus_one_ea(lambda x : Jump(x, k), termination_criterion, n)
            f_values.append(Jump(x, k))
            if x.bit_values == [1]*n:
                success_count += 1
                t_s.append(t)
        results_one_plus_one_ea_Jump[(n, k)] = (success_count/10, np.mean(t_s) if success_count > 0 else None,
                                            np.mean(f_values), np.std(f_values))


## Results

In [127]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown

def compile_all_results():
    # 1. Define the mapping of your notebook variables to readable labels
    # Format: (Variable Name String, Algorithm Label, Function Label, Parameter Name)
    result_sources = [
        # --- OneMax ---
        ("results_sig_cga_simplified_One_Max", "SigCGA (Simplified)", "OneMax", "epsilon"),
        ("results_sig_cga_original_One_Max",   "SigCGA (Original)",   "OneMax", "epsilon"),
        ("results_cga_One_Max",                "Standard CGA",        "OneMax", "K"),
        ("results_one_plus_one_ea_One_Max",    "1+1 EA",              "OneMax", "N/A"),
        
        # --- LeadingOnes ---
        ("results_sig_cga_simplified_LeadingOnes", "SigCGA (Simplified)", "LeadingOnes", "epsilon"),
        ("results_sig_cga_original_LeadingOnes",   "SigCGA (Original)",   "LeadingOnes", "epsilon"),
        ("results_cga_LeadingOnes",                "Standard CGA",        "LeadingOnes", "K"),
        ("results_one_plus_one_ea_LeadingOnes",    "1+1 EA",              "LeadingOnes", "N/A"),
        
        # --- Jump ---
        ("results_sig_cga_simplified_Jump", "SigCGA (Simplified)", "Jump", "epsilon"),
        ("results_sig_cga_original_Jump",   "SigCGA (Original)",   "Jump", "epsilon"),
        ("results_cga_Jump",                "Standard CGA",        "Jump", "K"),
        ("results_one_plus_one_ea_Jump",    "1+1 EA",              "Jump", "N/A"),
    ]

    all_data = []

    # 2. Iterate through the defined sources
    for var_name, algo, func, param_label in result_sources:
        # Safely get the variable from global scope; defaults to empty dict if not run yet
        data_dict = globals().get(var_name, {})
        
        if not data_dict:
            continue

        for key, val in data_dict.items():
            # Unpack values (Success, Runtime, Fit, Std)
            success_rate, mean_runtime, mean_fit, std_fit = val
            
            # Unpack Keys: Handle the variations in key structure (n) vs (n, param) vs (n, param, k)
            n, param_val, jump_k = None, None, None
            
            # Case 1: Just n (e.g., 1+1 EA OneMax)
            if isinstance(key, int):
                n = key
                param_val = "N/A"
                jump_k = "N/A"
            
            # Case 2: Tuple length 2 (e.g., CGA OneMax (n, K) OR 1+1 EA Jump (n, k))
            elif isinstance(key, tuple) and len(key) == 2:
                if func == "Jump" and algo == "1+1 EA": 
                    # Special case for EA on Jump: key is (n, k)
                    n, jump_k = key
                    param_val = "N/A"
                else:
                    # Standard case: key is (n, param)
                    n, param_val = key
                    jump_k = "N/A"
            
            # Case 3: Tuple length 3 (e.g., CGA Jump (n, K, k))
            elif isinstance(key, tuple) and len(key) == 3:
                n, param_val, jump_k = key

            # formatting parameter value
            if isinstance(param_val, float):
                param_val = round(param_val, 2)

            all_data.append({
                "Algorithm": algo,
                "Function": func,
                "n": n,
                "Parameter Type": param_label,
                "Parameter Value": param_val,
                "Jump k": jump_k,
                "Success Rate": success_rate,
                "Mean Runtime": mean_runtime,
                "Mean Fitness": mean_fit
            })

    # 3. Create DataFrame
    if not all_data:
        return pd.DataFrame()

    df = pd.DataFrame(all_data)
    
    # Reorder columns for logical reading
    cols = ["Function", "Algorithm", "n", "Parameter Type", "Parameter Value", 
            "Jump k", "Success Rate", "Mean Runtime", "Mean Fitness"]
    
    # Sort for cleaner display
    df = df[cols].sort_values(by=["Function", "Algorithm", "n", "Parameter Value"])
    
    return df

# Execute and Display
df_summary = compile_all_results()

if not df_summary.empty:
    display(Markdown("### Complete Experiment Summary"))
    # Apply styling for better readability
    # (Hiding index, formatting floats)
    display(df_summary.style.format({
        "Mean Runtime": "{:.1f}", 
        "Mean Fitness": "{:.2f}", 
        "Success Rate": "{:.1%}"
    }).hide(axis="index"))
else:
    print("No results found. Please ensure you have run the experiment cells in Task 6.")

### Complete Experiment Summary

Function,Algorithm,n,Parameter Type,Parameter Value,Jump k,Success Rate,Mean Runtime,Mean Fitness
Jump,1+1 EA,100,N/A,N/A,2,20.0%,1985.5,100.40
Jump,1+1 EA,100,N/A,N/A,3,0.0%,nan,100.00
Jump,1+1 EA,300,N/A,N/A,2,0.0%,nan,299.30
Jump,1+1 EA,300,N/A,N/A,3,0.0%,nan,299.70
Jump,SigCGA (Original),100,epsilon,9,2,0.0%,nan,55.10
Jump,SigCGA (Original),100,epsilon,9,3,0.0%,nan,55.10
Jump,SigCGA (Original),100,epsilon,12,2,0.0%,nan,53.60
Jump,SigCGA (Original),100,epsilon,12,3,0.0%,nan,54.00
Jump,SigCGA (Original),300,epsilon,9,2,0.0%,nan,156.80
Jump,SigCGA (Original),300,epsilon,9,3,0.0%,nan,160.00
